<a href="https://colab.research.google.com/github/Requenamar3/Machine-Learning/blob/main/Simple_Multiple_LinearReg_Spring25_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Simple and Multiple Linear Regression with Python

__Relevant textbook sections:__

Chapter 3: Linear Regression

3.1. Simple Linear Regression

3.2. Multiple Linear Regression

There are several options to run a regression analysis in Python. You can use Scikit-learn, Statsmodels, SciPy, and probably other packages.

We are going to learn how to do a linear regression analysis using the Scikit-learn package.

A common question among students who are taking this course and just took the Statistic with R course: Why doesn't Scikit-Learn provide p-values?

Scikit-learn's LinearRegression is designed for efficient computation and predictive performance, not hypothesis testing or model diagnostics.

**Statistical significance is not essential for many machine learning tasks, as the focus is typically on predictive accuracy rather than understanding the underlying relationships between variables.**

In [1]:
# Importing some of the required packages

import pandas as pd
import numpy as np

We are going to use the Boston dataset.

I chose to use this dataset to illustrate linear regression in Python because you are already familiar with it (this was the main dataset we used to learn regression in R).

__REMINDER !!!__ This is NOT the ethically compromised Boston dataset. It is a different one.

<br>

1) Download the "Boston.csv" from Canvas.

2) Upload the "Boston.csv" to your Google Drive. I recommend you to create a folder called "Datasets_CAP4631C" in your Google Drive to keep all the datasets for this class.

3) Mount Google Drive in Colab by running the next code cell.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_boston = "/content/sample_data/Boston.csv"

In [4]:
boston_df = pd.read_csv (path_boston)

In [5]:
boston_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   indus    506 non-null    float64
 3   chas     506 non-null    int64  
 4   nox      506 non-null    float64
 5   rm       506 non-null    float64
 6   age      506 non-null    float64
 7   dis      506 non-null    float64
 8   rad      506 non-null    int64  
 9   tax      506 non-null    int64  
 10  ptratio  506 non-null    float64
 11  lstat    506 non-null    float64
 12  medv     506 non-null    float64
dtypes: float64(10), int64(3)
memory usage: 51.5 KB


In [6]:
boston_df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


**How to evaluate the quality (performance) of an equation?**

**The Stat approach and the ML approach**

**Stat approach**

Evaluate the performance of the equation only on the training data, but make sure to adjust the evaluation for overfitting.

How to adjust for overfitting?

Use adjusted R squared, Residuals Standard Error (RSE), Cp, BIC, ... (or any metric that adjusts for overfitting).

**ML aproach**

Evaluate the performance of the equation on test data by using a single test-training split or cross-validation.

**Standardize the predictors before applying linear regression?**

It is not neccesary to do so when applying linear regression (it is not wrong if you do it, it is just not a neccesity). Read ISLP book, page 242 for an explanation.

Other regression techniques do require the standardization of the predictors (KNN regression, Ridge Regression, etc).

See this Minitab post for more ideas about this issue:

https://blog.minitab.com/en/adventures-in-statistics-2/when-is-it-crucial-to-standardize-the-variables-in-a-regression-model



### Simple Linear Regression: Stat approach

Let's do the same example we did in the Statistics with R class. That is, the linear regression of __medv__ versus __lstat__.

Before applying regression, let's do the usual preliminary analysis. Specifically, let's compute the __correlation coefficient__ between medv and lstat and do a __scatterplot__ between these two variables.

One of the many options we have to compute the correlation coefficient in Python is to use the corr() method from the Pandas package.

In [ ]:
boston_df[['lstat', 'medv']].corr()

In [ ]:
# If you only want to get the correlation value

boston_df[['lstat', 'medv']].corr().iloc [0, 1]

**Scatterplot**

Do a scatterplot between lstat and medv using using the functions from the **matplotlib package**.

**About matplotlib**: Quoting from **w3schools** ... "Most of Matplotlib utilities lies under the pyplot submodule"

That's why we import Matplotlib and the pyplot submodule.

In [ ]:
import matplotlib.pyplot as plt

We are going to use _plt.scatter()_ method since it is the conventional way of doing a scatter plot.

Comment: If your aren't trying to do a sophisticated scatter plot, like in our case, you could also use _plt.plot()_ to do a scatterplot

In [ ]:
plt.figure(figsize=(8, 8)) # Play around with different values in this tuple to get different figure sizes

plt.scatter(boston_df['lstat'], boston_df['medv'], c='blue')

plt.ylabel("medv")

plt.xlabel("lstat")

plt.title ("Median house values VS % of houses with low SES in the neighborhood")

# The next line is optional. Use if you want to add more marks on the y axis (plt.xticks for x axis)

plt.yticks(np.arange(boston_df['medv'].min(), boston_df['medv'].max()+1, 5))

plt.show()

 **Obtain the linear regression model**

We start by importing the 'LinearRegression' function from the scikit-learn module 'linear_model'.

In [ ]:
from sklearn.linear_model import LinearRegression

Any method part of scikit-learn REQUIRES that we use a __two dimensional object__ to store the predictor(s) values.

So, what do we do in this case where we only have one predictor? To store the values of one predictor, we only need need a one-dimensional array. Thus, what do we do?

We transform the array with the values of the predictor in a two dimensional array where the number of columns equals 1 and the number of rows equals the number of elements in the array.

We can use the reshape() method to do that.

In [ ]:
np.array(boston_df['lstat'])

In [ ]:
# 'lstat' as a one dimensional array

print ( np.array(boston_df['lstat']).ndim )

print ( np.array(boston_df['lstat']).shape )


In [ ]:
# 'lstat' as a two dimensional array with m rows and 1 column

np.array(boston_df['lstat']).reshape(-1, 1)

In [ ]:
np.array(boston_df['lstat']).reshape(-1,1).ndim

In [ ]:
np.array(boston_df['lstat']).reshape(-1,1).shape

Now that we undertand how to transform the predictor into a two dimensional array, we create a variable to store the predictor and another one to store the outcome.

In [ ]:
X_lstat = np.array(boston_df['lstat']).reshape(-1,1)

In [ ]:
# No need to convert into NumPy or reshape the column with the outcome variable

y = boston_df['medv']

Now we invoke the fit() method, to fit a linear regression model using the predictor and outcome data.

In [ ]:
# I create the 'reg_out_lstat' variable to easily access the regression results

reg_out_lstat = LinearRegression().fit(X_lstat, y)

In [ ]:
# Intercept of the regression equation

reg_out_lstat.intercept_

In [ ]:
# Slope of the regression equation

reg_out_lstat.coef_

**Evaluate the quality of the simple linear regression equation**

**R squared**

Scikit-learn has a function to compute R squared. Let's import it:

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
# Store the predictions of y given by the equation in an array.
# This is optional but recommmended

y_pred_lstat= reg_out_lstat.predict(X_lstat)

Notice that in scikit-learn, when we use predict(), we need to pass the values of the predictor as an argument.

In [ ]:
r2_score(y, y_pred_lstat)

**RSE**

Given that scikit learn does not have a function to compute RSE, we will define our own function to compute. Defining a function is NOT mandatory, but it keeps the code cleaner and facilitates re-using the function.

As a reminder, this is the formula of RSE:

sqrt (SSE / (n-p-1) )

where,

SSE: Sum of Square of Errors

n: sample size

p: number of predictors

In [ ]:
def rse_calculator (y_actual, y_predicted, p):

  rse_value = np.sqrt ( np.sum((y_actual - y_predicted)**2) / (y_actual.size - p -1) )

  return np.round (rse_value, 4)

In [ ]:
rse_calculator(y, y_pred_lstat, 1)

Is RSE low (= good)?

In [ ]:
# Coefficient of variation
# RSE / mean of Y

rse_calculator(y, y_pred_lstat, 1) / np.mean (y)

__How to make predictions with the estimated equation?__

For example, let's predict the values of medv based on the regression equation for five new neighborhoods. These are the values of lstat for these five new neighborhoods:

4.5, 5, 7, 8.5, and 9.3

In [ ]:
reg_out_lstat.predict( np.array( [4.5, 5, 7, 8.5, 9.3] ).reshape (-1, 1) )

**Plot of Residuals versus Predicted values**

First, let's repeat the previous scatter plot of medv VS LSTAT, but let's add the regression line to it this time.

In [ ]:
plt.figure(figsize=(8, 8)) # Play around with values in this tuple to get different figure sizes

plt.scatter(boston_df['lstat'], boston_df['medv'], c='blue')

plt.ylabel("medv")

plt.xlabel("lstat")

plt.title ("Median house values VS % of houses with low SES in the neighborhood")

plt.yticks(np.arange(boston_df['medv'].min(), boston_df['medv'].max()+1, 5))

# This is the additional statement needed to plot the regression line in the scatterplot

plt.plot(boston_df['lstat'], y_pred_lstat, c='red', ls='-') # ls means line style. Use '-' to get a solid line. Use '--' for a dashed line

plt.show()

In [ ]:
# Let's compute the residuals and store them in an array.
# residuals = y actual - y predicted

residuals_lstat = y - y_pred_lstat

In [ ]:
plt.scatter(y_pred_lstat, residuals_lstat,c='blue')

plt.xlabel("Predicted y") # Predicted values of medv obtained from the equation
plt.ylabel("Residuals")
plt.axhline(0, c='red',ls='--')
plt.show()

### Multiple Linear Regression: Stat approach

We are still using the Boston dataset.

As an illustration, let's do the multiple linear regression of medv VS lstat and rm.

In [ ]:
X_lstat_rm= boston_df[['lstat','rm']]

# no need to convert to a two dimensional array because boston_df[['lstat','rm']]  is a dataframe (data frame have two dimensions)

In [ ]:
# I create the 'reg_out_lstat_rm' variable to easily access the regression results

reg_out_lstat_rm = LinearRegression().fit(X_lstat_rm, y)

In [ ]:
# Intercept of the regression equation

reg_out_lstat_rm.intercept_

In [ ]:
# Coefficients of the multiple regression equation

reg_out_lstat_rm.coef_

The next code cell shows you how to create a data frame with all the coefficients displayed in a nice way:

In [ ]:
# to retrieve the names of the predictors used in the model

reg_out_lstat_rm.feature_names_in_

In [ ]:
coef_values = np.concatenate(([reg_out_lstat_rm.intercept_], reg_out_lstat_rm.coef_))
# need to convert the intercept value into a list or array bf using it in concatenate.

column_names = np.concatenate((['Intercept'], reg_out_lstat_rm.feature_names_in_))

# Create a data frame to display the results

coefficients_df = pd.DataFrame({'Coefficient Name': column_names, 'Coefficient Value': coef_values})

print(coefficients_df)

**Evaluate the quality of a multiple linear regression equation**

**R squared**

In [ ]:
y_pred_lstat_rm= reg_out_lstat_rm.predict(X_lstat_rm)

In [ ]:
r2_score ( y, y_pred_lstat_rm)

**Adjusted R squared**

If you want to use a classic statistical approach to compare a multiple linear equation with another one with a different number of predictors, you should adjust for overfitting. Adjusted R squared is one of the many metrics that does that (RSE does it too).

In R, the function used to apply regression gives us both R squared and adjusted R squared. Scikit-learn gives us R squared but not adjusted R squared.

To compute adjusted r squared, we are going t write our own function based on this formula:

adj r squared = 1 - (1 -  r squared) * ((n - 1)/(n-p-1))

In [ ]:
def adj_r2_calculator (r2_value, n, p):

  adj_r2_value = 1 - (1- r2_value) * ( (n - 1) / (n - p - 1) )

  return np.round (adj_r2_value, 4)

In [ ]:
# 'shape' gives us the sample size (using index 0) and the number of predictors (using index 1)

adj_r2_calculator (r2_score ( y, y_pred_lstat_rm), X_lstat_rm.shape[0], X_lstat_rm.shape[1])

**RSE**

RSE also adjusts for overfitting; thus, it can be used to compare equations with different number of predictors.

In [ ]:
rse_calculator(y, y_pred_lstat_rm, X_lstat_rm.shape[1])

**How to use Adjusted R Squared and RSE to compare equations with different number of predictors?**

Let's compare the equation that only uses 'lstat' as predictor with the one that uses 'lstat' and 'rm'.

__Fragments from the R class:__

*In this course, unless I tell you otherwise, we will consider that an equation is better than another one only if the **increase in Adjusted R Squared is deemed to be practically significant**. When is an increase in Adjusted R squared practically significant? If it is at least a 5% increase  (**% increase in Adj R squared >= 5%**)*

*In some context, people are taught that as long as Adjusted R Squared increases, even if only by a little bit, that should be taken as a sign of improvement. However, such an approach is too naive and simplistic.*

*In this course, unless I tell you otherwise, we will consider that an equation is better than  another one only if the **decrease in RSE is deemed to be practically significant**. When is decrease in RSE practically significant? If it is at least a 5% decrease  (**% decrease in RSE >= 5%**).*

*In some context, people are taught that as long as RSE decreases, even if only by a little bit, that should be taken as a sign of improvement. However, such an approach is too naive and simplistic.*

Let's compute the % increase in adjusted R squared from model 1 (lstat only) to model 2 (lstat and rm)

In [ ]:
adj_r_sq_lstat = adj_r2_calculator (r2_score ( y, y_pred_lstat), X_lstat.shape[0], X_lstat.shape[1])

In [ ]:
adj_r_sq_lstat_rm = adj_r2_calculator (r2_score ( y, y_pred_lstat_rm), X_lstat_rm.shape[0], X_lstat_rm.shape[1])

In [ ]:
# % increase in adjusted R squared

( (adj_r_sq_lstat_rm -  adj_r_sq_lstat)/ adj_r_sq_lstat )*100

So, what's the conclusion???

Let's compute the % decrease in RSE from model 1 (lstat only) to model 2 (lstat and rm)

In [ ]:
rse_lstat = rse_calculator(y, y_pred_lstat, X_lstat.shape[1])

In [ ]:
rse_lstat_rm = rse_calculator(y, y_pred_lstat_rm, X_lstat_rm.shape[1])

In [ ]:
( (rse_lstat_rm - rse_lstat)/rse_lstat ) * 100

Conclusion???

## A second example of Linear Regression with Python: the prostate dataset.

Download the prostate.csv file from Canvas. Read it as a Pandas data frame and call it _prostate_df_

Info about this dataset (from the Elements of Statistical Learning):

_"The data for this example come from a study by Stamey et al. (1989). They
examined the correlation between the level of prostate-specific antigen (lpsa) and
a number of clinical measures in men who were about to receive a radical
prostatectomy. The variables are log cancer volume (lcavol), log prostate
weight (lweight), age, log of the amount of benign prostatic hyperplasia
(lbph), seminal vesicle invasion (svi), log of capsular penetration (lcp),
Gleason score (gleason), and percent of Gleason scores 4 or 5 (pgg45)."_

__The outcome variable is _lpsa_.__

In [ ]:
prostate_data_path = "/content/drive/MyDrive/CAP 4631C/Datasets_CAP4631C/prostate.csv"

In [ ]:
prostate_df = pd.read_csv(prostate_data_path)

In [ ]:
prostate_df.info()

In [ ]:
# Drop the last column. Not useful

prostate_df.drop (['train'],axis=1, inplace= True)

In [ ]:
prostate_df.info()

Find the best predictor to use in a simple LR model (a model with only one variable)

In [ ]:
prostate_df.corr()

In [ ]:
prostate_df.corr()['lpsa']

Use the statistical approach and RSE to compare two equations:

The equation that only includes the predictor you chose in the previous step.

The equation that includes the predictor you chose in the previous step AND 'lcp'.

CONTINUE WORKING INDEPENDENTLY FROM HERE ON!